# Final Project
## CSCI 5845: Natural Language Processing

### Student Information
- **Name**: Lohit Akkineni
- **Instructor**: Willbert Hawley

### Project Overview
This project involves building and evaluating several neural network models to perform language identification. The dataset consists of approximately 10,000 entries covering 17 different languages. The goal is to accurately classify the language of each text entry using three distinct types of neural network architectures: Dense Neural Networks, Convolutional Neural Networks (CNN), and Long Short-Term Memory (LSTM) Networks.

### Objectives
- To apply and compare different neural network architectures for the task of language identification.
- To evaluate the models based on their accuracy, training time, and generalizability on test data.
- To determine the most effective model architecture for this specific application.

### Methodology
The project will follow these steps:
1. Data Preprocessing: Cleaning, tokenizing, and vectorizing the text data.
2. Model Building: Constructing three different models with specific architectures.
3. Training and Evaluation: Training the models on the dataset and evaluating their performance.
4. Comparison and Analysis: Comparing the models based on various performance metrics and providing a comprehensive analysis.

This project aims to leverage advanced techniques in natural language processing to solve a practical and relevant challenge in the field.


# Loading the dataset

In [1]:
import pandas as pd

# Load the dataset
data_path = "languages.csv"
data = pd.read_csv(data_path)

# Display the first few rows of the dataset to understand its structure
data.head(10)


C:\Users\akkin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\akkin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
5,"[2] In ancient philosophy, natura is mostly us...",English
6,"[3][4] \nThe concept of nature as a whole, the...",English
7,During the advent of modern scientific method ...,English
8,"[5][6] With the Industrial revolution, nature ...",English
9,"However, a vitalist vision of nature, closer t...",English


# Data Preprocessing

In [2]:
data['Text'] = data['Text'].str.lower()


In [3]:
import string
data['Text'] = data['Text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))


In [4]:
data.dropna(inplace=True)  # Remove rows with missing values


# Vectoriztion

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(data['Text'])


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize the tokenizer with a specified maximum number of words (optional)
tokenizer = Tokenizer(num_words=10000)  # Adjust `num_words` as necessary

# Assuming 'data' is your DataFrame and it has a column 'Text' that contains the text entries
tokenizer.fit_on_texts(data['Text'])


In [8]:
# Convert text to sequences of integers
X_train_seq = tokenizer.texts_to_sequences(data['Text'])

# Import the pad_sequences method
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad sequences to ensure uniform input size; adjust 'maxlen' as needed based on your data analysis
X_train_padded = pad_sequences(X_train_seq, maxlen=200)  # 'maxlen' should be chosen based on the length distribution of your texts


In [9]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Initialize and fit the tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['Text'])

# Convert text to sequences and pad them
X_train_seq = tokenizer.texts_to_sequences(data['Text'])
X_train_padded = pad_sequences(X_train_seq, maxlen=200)



In [10]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Instantiate the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels to integer encoded
y_encoded = label_encoder.fit_transform(data['Language'])

# Convert integer encoded labels to one-hot encoded
y_one_hot = to_categorical(y_encoded)


# Model building and training

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model_dense = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_padded.shape[1],)),
    Dense(y_one_hot.shape[1], activation='softmax')
])

# Compile the model
model_dense.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Dense model
history_dense = model_dense.fit(X_train_padded, y_one_hot, epochs=10, validation_split=0.1)


Epoch 1/10


291/291 [==============================] - 2s 4ms/step - loss: 270.6005 - accuracy: 0.1034 - val_loss: 4043.5171 - val_accuracy: 0.0106
Epoch 2/10
291/291 [==============================] - 1s 3ms/step - loss: 138.1772 - accuracy: 0.1363 - val_loss: 5446.1895 - val_accuracy: 0.0029
Epoch 3/10
291/291 [==============================] - 1s 3ms/step - loss: 95.9905 - accuracy: 0.1688 - val_loss: 5512.4751 - val_accuracy: 0.0029
Epoch 4/10
291/291 [==============================] - 1s 3ms/step - loss: 73.2890 - accuracy: 0.1934 - val_loss: 5254.6172 - val_accuracy: 0.0019
Epoch 5/10
291/291 [==============================] - 1s 3ms/step - loss: 60.9427 - accuracy: 0.2067 - val_loss: 4760.2739 - val_accuracy: 0.0029
Epoch 6/10
291/291 [==============================] - 1s 3ms/step - loss: 50.7247 - accuracy: 0.2227 - val_loss: 4384.0771 - val_accuracy: 0.0077
Epoch 7/10
291/291 [==============================] - 1s 2ms/step - loss: 42.0632 - accuracy: 0.2366 - val_loss: 3876.05

In [12]:
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D

# Define the model
model_cnn = Sequential([
    Embedding(input_dim=10000, output_dim=50, input_length=200),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dense(y_one_hot.shape[1], activation='softmax')
])

# Compile the model
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
# CNN model
history_cnn = model_cnn.fit(X_train_padded, y_one_hot, epochs=10, validation_split=0.1)

Epoch 1/10
291/291 [==============================] - 4s 11ms/step - loss: 1.2798 - accuracy: 0.6160 - val_loss: 9.8714 - val_accuracy: 0.1596
Epoch 2/10
291/291 [==============================] - 3s 10ms/step - loss: 0.1354 - accuracy: 0.9665 - val_loss: 10.8430 - val_accuracy: 0.1605
Epoch 3/10
291/291 [==============================] - 3s 10ms/step - loss: 0.0721 - accuracy: 0.9794 - val_loss: 11.8640 - val_accuracy: 0.1615
Epoch 4/10
291/291 [==============================] - 3s 10ms/step - loss: 0.0618 - accuracy: 0.9805 - val_loss: 12.2929 - val_accuracy: 0.1605
Epoch 5/10
291/291 [==============================] - 3s 10ms/step - loss: 0.0574 - accuracy: 0.9814 - val_loss: 13.3916 - val_accuracy: 0.1615
Epoch 6/10
291/291 [==============================] - 3s 10ms/step - loss: 0.0565 - accuracy: 0.9822 - val_loss: 13.9524 - val_accuracy: 0.1605
Epoch 7/10
291/291 [==============================] - 3s 10ms/step - loss: 0.0560 - accuracy: 0.9823 - val_loss: 14.0647 - val_accuracy: 

In [13]:
from tensorflow.keras.layers import LSTM

# Define the model
model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=50, input_length=200),
    LSTM(64),
    Dense(128, activation='relu'),
    Dense(y_one_hot.shape[1], activation='softmax')
])

# Compile the model
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
# LSTM model
history_lstm = model_lstm.fit(X_train_padded, y_one_hot, epochs=10, validation_split=0.1)

Epoch 1/10
291/291 [==============================] - 22s 67ms/step - loss: 1.4575 - accuracy: 0.5352 - val_loss: 9.4746 - val_accuracy: 0.1460
Epoch 2/10
291/291 [==============================] - 20s 67ms/step - loss: 0.2482 - accuracy: 0.9346 - val_loss: 10.2017 - val_accuracy: 0.1383
Epoch 3/10
291/291 [==============================] - 20s 67ms/step - loss: 0.1156 - accuracy: 0.9728 - val_loss: 11.2149 - val_accuracy: 0.1615
Epoch 4/10
291/291 [==============================] - 20s 67ms/step - loss: 0.0825 - accuracy: 0.9775 - val_loss: 10.0033 - val_accuracy: 0.1605
Epoch 5/10
291/291 [==============================] - 19s 66ms/step - loss: 0.0706 - accuracy: 0.9800 - val_loss: 10.8617 - val_accuracy: 0.1586
Epoch 6/10
291/291 [==============================] - 19s 67ms/step - loss: 0.0650 - accuracy: 0.9811 - val_loss: 11.1467 - val_accuracy: 0.1847
Epoch 7/10
291/291 [==============================] - 19s 67ms/step - loss: 0.0609 - accuracy: 0.9820 - val_loss: 10.3457 - val_acc

## Summary of Model Performances and Evaluations

### Dense Neural Network:
- **Performance**: The Dense model displayed relatively low accuracy with excessively high validation losses, indicating poor generalization and a potential underfitting to the complexities of the language identification task.
- **Possible Improvements**: Enhancing the model's complexity, using different vectorization strategies, and incorporating regularization techniques like dropout might improve its performance.

### Convolutional Neural Network (CNN):
- **Performance**: Exhibited higher accuracy, indicating effective spatial pattern recognition in text data. However, the model's high validation losses suggest significant overfitting.
- **Possible Improvements**: Implementing dropout or spatial dropout layers and experimenting with the number of filters or layers could help reduce overfitting and capture more nuanced features.

### Long Short-Term Memory (LSTM):
- **Performance**: Achieved good accuracy, similar to CNN, but also showed signs of overfitting as indicated by high validation losses. Its architecture is well-suited for capturing long-term dependencies in sequence data, beneficial for processing language.
- **Possible Improvements**: Including LSTM-specific dropout strategies and adjusting the number of units or layers could enhance model robustness and performance.

## Recommendations and Further Steps:
- **Model Selection**: Given their superior performance, both CNN and LSTM are recommended over the Dense model. The choice between CNN and LSTM would depend on specific application needs regarding computational resources and response time.
- **Handling Overfitting**: For both CNN and LSTM, applying more rigorous regularization and possibly simplifying the model architectures are crucial steps towards better generalization.
- **Further Experimentation**: Exploring hybrid models that combine the strengths of CNNs and LSTMs could potentially offer improvements in learning both spatial and temporal features in the text data.

## Concluding Thoughts:
All models exhibit potential but also a tendency towards overfitting, highlighting the importance of a balanced approach in model training and architecture design. Fine-tuning these models based on the insights from their performance could lead to more robust solutions for the language identification task.

For further analysis or implementation guidance, additional support is available as needed. This documentation ensures clarity in the project's progression and serves as a basis for future enhancements.
